# AsyncIO
通过CLI的方式访问设备的时候，每一命令都是一个有阻的操作。当需要对很多设备执行相同或不同的命令的时候，需要消耗很多的时间才能完成。

比如通过bash 的方式来访问每台设备的配置信息

In [6]:
!sshpass -p admin ssh -l admin 172.20.3.101 show ipv4 interface brief




IMPORTANT:  READ CAREFULLY
Welcome to the Demo Version of Cisco IOS XRv (the "Software").
The Software is subject to and governed by the terms and conditions
of the End User License Agreement and the Supplemental End User
License Agreement accompanying the product, made available at the
time of your order, or posted on the Cisco website at
www.cisco.com/go/terms (collectively, the "Agreement").
As set forth more fully in the Agreement, use of the Software is
strictly limited to internal use in a non-production environment
solely for demonstration and evaluation purposes.  Downloading,
installing, or using the Software constitutes acceptance of the
Agreement, and you are binding yourself and the business entity
that you represent to the Agreement.  If you do not agree to all
of the terms of the Agreement, then Cisco is unwilling to license
the Software to you and (a) you may not download, install or use the
Software, and (b) you may return the Software as more fully set forth
in the 

### 访问多台设备信息
在 Jupyter lab 中打开 terminal 执行以下代码：

```

  for x in {101..115};
   do sshpass -p admin ssh -l admin 172.20.3.${x} show ipv4 interface brief;
  done
  
```
bash 会登录到每一台设备上，然后获取设备的信息。当设备非常多的时，需要执行很久的时间


# 使用Pexpect登录设备

In [24]:
#!/usr/bin/env python
#coding: utf-8
import pexpect
from pexpect import EOF, TIMEOUT

def ssh_connect(username, address, port):
    ssh_command = 'ssh  -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -l %s %s -p %d' % (
        username, address, port)
    return pexpect.spawn(ssh_command)


class Device(object):

    def __init__(self, device):
    
        self.hostname = device.get("hostname")
        self.mgt_ip = device.get("mgt_ip")
        self.username = device.get("username")
        self.password = device.get("password")
        self.port = device.get("port", 22)
        self.expect_list = []

    def connect(self, timeout=30):
        self.c = ssh_connect(self.username, self.mgt_ip, self.port)
        self.c.delaybeforesend = 0.10
        return self.c

    def login(self, prompt=r"[>|#|$]\s?$"):
        self.expect_list = []
        self.expect_list.append(r"(?i)username[:]?\s*$")
        self.expect_list.append(r"(?i)login[:]?\s*$")
        self.expect_list.append(r"(?i)password[:]?\s*$")
        self.expect_list.append(prompt)
        for _ in range(0, 3):
            result = []
            try:
                i = self.c.expect(self.expect_list, timeout=5)
                result.append(i)
                result.append(str(self.c.before))
                result.append(str(self.c.after))
                if i < 2:
                    self.c.sendline(self.username)
                elif i == 2:
                    self.c.sendline(self.password)
                elif i == 3:
                    break
            except EOF:
                break
            except TIMEOUT:
                print("connect to %s timeout" %self.hostname)
                break
        
        if result[0] < 3:
            print("username or password error")
            return result
        
        self._set_terminal_length_zero()
        return result

    def _set_terminal_length_zero(self):
        pass

    def get_config(self):
        pass
    
    def logout(self):
        if self.c:
            self.c.terminate()

    def __del__(self):
        self.logout()

        
class IOSXR(Device):
    def __init__(self, device):
        super(IOSXR, self).__init__(device)
        self.prompt = self.hostname + "[>|#]\s?"

    def login(self, prompt=""):
        if not prompt:
            prompt = self.prompt
        return super(IOSXR, self).login(prompt)

    def _set_terminal_length_zero(self):
        self.c.sendline("terminal length 0")
        try:
            i = self.c.expect(self.prompt)
        except EOF:
            pass
        except TIMEOUT:
            print("session timeout")

    def get_config(self):
        self.expect_list = []
        self.expect_list.append(self.prompt)
        result = []
        self.c.sendline("show running-config")
        try:
            i = self.c.expect(self.expect_list, timeout=5)
            if i == 0:
                result.append(i)
                result.append(str(self.c.before))
                result.append(str(self.c.after))
        except EOF:
            pass
        except TIMEOUT:
            print("session timeout")
        return result

In [25]:
xrv1 = {"hostname":"xrv1", 
     "mgt_ip":"172.20.3.101",
     "username":"admin",
     "password":"admin",
     "port": 22}

d = IOSXR(xrv1)
d.connect()
d.login()
d.get_config()

[0,
 "b'show running-config\\r\\n\\rFri Aug 24 02:53:01.542 UTC\\r\\nBuilding configuration...\\r\\n!! IOS XR Configuration 6.5.1.31I\\r\\n!! Last configuration change at Sat Aug 11 05:22:38 2018 by admin\\r\\n!\\r\\nhostname xrv1\\r\\ninterface Loopback0\\r\\n ipv4 address 99.1.0.1 255.255.255.255\\r\\n!\\r\\ninterface MgmtEth0/0/CPU0/0\\r\\n ipv4 address 172.20.3.101 255.255.0.0\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/0\\r\\n ipv4 address 99.1.2.1 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/1\\r\\n ipv4 address 99.1.2.5 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/2\\r\\n ipv4 address 99.1.2.9 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/3\\r\\n ipv4 address 99.1.2.13 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/4\\r\\n ipv4 address 99.1.2.17 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/5\\r\\n ipv4 address 99.1.2.21 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/6\\r\\n ipv4 address 99.1.2.25 255.2

## 修改为AsyncIO 的代码

当jupyter lab 使用了tornado 5.0 后，在启动jupyter 后已经运行了一个event_loop。

我们需要启动一个新的event_loop 才能完成使用python 的asyncio。

https://github.com/jupyter/notebook/issues/3397

In [1]:
import pexpect
from pexpect import EOF, TIMEOUT

def ssh_connect(username, address, port):
    ssh_command = 'ssh  -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -l %s %s -p %d' % (
        username, address, port)
    return pexpect.spawn(ssh_command)


class Device(object):

    def __init__(self, device):
    
        self.hostname = device.get("hostname")
        self.mgt_ip = device.get("mgt_ip")
        self.username = device.get("username")
        self.password = device.get("password")
        self.port = device.get("port", 22)
        self.expect_list = []

    def connect(self, timeout=30):
        self.c = ssh_connect(self.username, self.mgt_ip, self.port)
        self.c.delaybeforesend = 0.10
        return self.c
    # 修改 async 函数
    async def login(self, prompt=r"[>|#|$]\s?$"):
        self.expect_list = []
        self.expect_list.append(r"(?i)username[:]?\s*$")
        self.expect_list.append(r"(?i)login[:]?\s*$")
        self.expect_list.append(r"(?i)password[:]?\s*$")
        self.expect_list.append(prompt)
        for _ in range(0, 3):
            result = []
            try:
                # i = self.c.expect(self.expect_list, timeout=5)
                # 需要等待一些时间才从设备上获取值。
                i = await self.c.expect(self.expect_list, timeout=5, async_=True)
                result.append(i)
                result.append(str(self.c.before))
                result.append(str(self.c.after))
                if i < 2:
                    self.c.sendline(self.username)
                elif i == 2:
                    self.c.sendline(self.password)
                elif i == 3:
                    break
            except EOF:
                break
            except TIMEOUT:
                print("connect to %s timeout" %self.hostname)
                break
        
        if result[0] < 3:
            print("username or password error")
            return result
        
        await self._set_terminal_length_zero()
        return result

    def _set_terminal_length_zero(self):
        pass

    def get_config(self):
        pass
    
    def logout(self):
        if self.c:
            self.c.terminate()

    def __del__(self):
        self.logout()

        
class IOSXR(Device):
    def __init__(self, device):
        super(IOSXR, self).__init__(device)
        self.prompt = self.hostname + "[>|#]\s?"

   # 需要等待一些时间才从设备上获取值。
    async def login(self, prompt=""):
        if not prompt:
            prompt = self.prompt
        await super(IOSXR, self).login(prompt)

    # 需要等待一些时间才从设备上获取值。
    async def _set_terminal_length_zero(self):
        self.c.sendline("terminal length 0")
        try:
            i = await self.c.expect(self.prompt, async_=True)
        except EOF:
            pass
        except TIMEOUT:
            print("session timeout")

    # 需要等待一些时间才从设备上获取值。
    async def get_config(self):
        self.expect_list = []
        self.expect_list.append(self.prompt)
        result = []
        self.c.sendline("show running-config")
        try:
            i = await self.c.expect(self.expect_list, timeout=5, async_=True)
            if i == 0:
                result.append(i)
                result.append(str(self.c.before))
                result.append(str(self.c.after))
        except EOF:
            pass
        except TIMEOUT:
            print("session timeout")
        return result

In [3]:
import asyncio

xrv1 = {"hostname":"xrv1", 
     "mgt_ip":"172.20.3.101",
     "username":"admin",
     "password":"admin",
     "port": 22}

xrv2 = {"hostname":"xrv2", 
     "mgt_ip":"172.20.3.102",
     "username":"admin",
     "password":"admin",
     "port": 22}

xrv3 = {"hostname":"xrv3", 
     "mgt_ip":"172.20.3.103",
     "username":"admin",
     "password":"admin",
     "port": 22}

async def get_config(device):
    con = IOSXR(device)
    con.connect()
    await con.login()
    result = await con.get_config()
    return result

# 当jupyter lab 使用了tornado 5.0 后，在启动jupyter 后已经运行了一个event_loop。
# 我们需要启动一个新的event_loop 才能完成使用python 的asyncio。
# https://github.com/jupyter/notebook/issues/3397


loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.run_until_complete(asyncio.gather(get_config(xrv1), 
                                       get_config(xrv2),
                                       get_config(xrv3)
                                      )
                       )




[[0,
  "b'show running-config\\r\\n\\rFri Aug 24 03:40:17.387 UTC\\r\\nBuilding configuration...\\r\\n!! IOS XR Configuration 6.5.1.31I\\r\\n!! Last configuration change at Sat Aug 11 05:22:38 2018 by admin\\r\\n!\\r\\nhostname xrv1\\r\\ninterface Loopback0\\r\\n ipv4 address 99.1.0.1 255.255.255.255\\r\\n!\\r\\ninterface MgmtEth0/0/CPU0/0\\r\\n ipv4 address 172.20.3.101 255.255.0.0\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/0\\r\\n ipv4 address 99.1.2.1 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/1\\r\\n ipv4 address 99.1.2.5 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/2\\r\\n ipv4 address 99.1.2.9 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/3\\r\\n ipv4 address 99.1.2.13 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/4\\r\\n ipv4 address 99.1.2.17 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/5\\r\\n ipv4 address 99.1.2.21 255.255.255.252\\r\\n!\\r\\ninterface GigabitEthernet0/0/0/6\\r\\n ipv4 address 99.1.2.25 255